In [ ]:
from telethon.sync import TelegramClient as TC
from telethon import TelegramClient
import json
from PIL import Image
import os

## Data gathering da Telegram in Python
La piattaforma Telegram rilascia due tipi di API per gli sviluppatori:
- **Bot API**: permette di connettere un bot alla piattaforma Telegram. I Bot non richiedono numero di telefono per il setup. La comunicazione avviene tra bot e la piattaforma avviene mediante HTTPs che offre una versione semplificata delle **Telegram API**
- **Telegram API**: API per creazione di un client Telegram customizzato. Per questo motivo è stata rilasciata la libreria **Telegram Database Library - TDLib**, che facilita l'implementazione di applicazione Telegram-based. La libreria si occupa della codifica/decodifica dei messaggi e della gestione dello storage locale, facilitando l'interazione con Telegram API.

In questa esperienza utilizzeremo Telegram API interfacciandosi mediante la libreria `telethon` in Python. Tale libreria ha come principale obiettivo facilitare la creazione di applicazione che possano interagire con Telegram.

#### Installazione
``` bash
pip install telethon
```

Per poter interagire con le Telegram API è necessario creare un'applicazione utilizzando API Development tools:
- Aprire la [pagina](https://my.telegram.org/) e loggarsi utilizzando il numero di telefono associato al vostro account in Telegram
- Selezionare _API Development tools_
- Creare una nuova applicazione specificando le informazioni essenziali (non è necessario completare i campi che richiedono un URL)

Il processo di creazione dell'applicazione vi restituisce due informazioni essenziali che utilizzeremo in seguito:
- **app_api_id**: id dell'applicazione creata
- **app_api_hash**: informazione confidenziale che corrisponde al `client secret` in OAuth2.0.

[Queste informazioni sono state salvate in un file JSON `telegram_api.json`]

Carichiamo le credenziali ricevute e salvate nel file `telegram_api.json`.

In [ ]:
credentials = json.load(open('telegram_api.json'))
client_id = credentials['app_api_id']
client_secret = credentials['app_api_hash']

In [ ]:
client_id = os.environ["APP_API_ID"]
client_secret = os.environ["APP_API_HASH"]

La classe principale per interfacciarsi con Telegram API è `TelegramClient`. Il costruttore richiede tre parametri:
- `session`: database file (SQLite) per salvare le informazioni di sessione.
- `api id`: identificativo dell'applicazione
- 'api hash': secret associato all'applicazione

Il primo parametro è importante in quanto consente di memorizzare le informazioni di sessione ed evita il continuo inserimento di codici temporanei di verifica dell'identità.

E' importante sottolineare il fatto che **solo una** sessione può essere aperta. In caso di richiesta di apertura di una seconda sessione sullo stesso DB si ottiene un errore. Per evitare questo inconveniente è consigliato definire l'oggetto `TelegramClient` in un context manager in modo che venga automaticamente chiusa la sessione, una volta terminato il blocco di codice contenuto nel context manager.

Un secondo elemento, molto importante, da tenere in considerazione è che la libreria utilizza richieste asincrone, quindi dovremmo utilizzare le keyword `async` e `await` per indicare che un metodo/oggetto è asincrono e che devo attendere l'esecuzione di un metodo asincrono - **coroutine**. Per approfondimenti sul tema AsyncIO in Python, clicca [qui](https://realpython.com/async-io-python/).

Nella pratica questo si traduce nell'utilizzo di `await` prima di invocare un metodo asincrono, la cui definizione è `async def`.

Costruiamo un oggetto `TelegramClient`. Se il client non è stato verificato, viene richiesto il numero di telefono associato al vostro account Telegram, sul quale, poi, riceverete un codice di validazione, anch'esso da inserire.

In [ ]:
client = TelegramClient('anon',client_id, client_secret)

All'interno di un context manager utilizziamo il metodo asincrono `get_me()` che restituisce un oggetto `User` corrispondente all'account Telegram associato all'applicazione in uso.

Lo username dell'account è associato all'attributo `username` dell'oggetto `User`

**Memo e suggerimento**: in questo caso definisco un context manager che gestisce un oggetto/metodo asincrono. La sintassi, in questo caso, è:
``` python
async with <identificatore oggetto>:
    <blocco codice>
```

In [ ]:
async with client:
    print((await client.get_me()).username)

matzigno


La classe `TelegramClient` rappresenta il principale punto di accesso per interagire con Telegram API mediante l'esecuzione dei metodi mostrati nelle API.

Per ottenere un elenco di tutti i metodi possiamo invocare il metodo
``` python
help(client)
```

In [ ]:
help(client)

Telethon utilizza il concetto di **entity** che si raggruppa un qualsiasi oggetto `User`, `Chat` or `Channel`. 

Se un metodo richiede un oggetto **entity_like** si deve fornire un elemento che può essere risolto in un'entità -> username, titolo esatto, oggetti User, Chat o Channel, numero di telefono.

Per ottenere un ID di un entità si devono utilizzare le stesse modalità per la ricerca dell'ID nell'applicazione mobile o web di Telegram. Una volta che la libreria "incontra“ per la prima volta un ID, lo salva nel DB - `SessionFiles`.

Per effettuare la ricerca di un'entita si utilizza il metodo `get_entity`.

Per esempio possiamo ottenere l'entità associata all'invito a partecipare al gruppo Telegram  che utilizzeremo durante questa esperienza - [https://t.me/+FG3x1SOuhJVhZTI0](https://t.me/+FG3x1SOuhJVhZTI0)

In [ ]:
async with client:
    group_test = await client.get_entity('https://t.me/+FG3x1SOuhJVhZTI0')
print(group_test.stringify())

Channel(
	id=2066577343,
	title='Someni Test Group',
	photo=ChatPhotoEmpty(
	),
	date=datetime.datetime(2024, 3, 19, 20, 43, 25, tzinfo=datetime.timezone.utc),
	creator=True,
	left=False,
	broadcast=False,
	verified=False,
	megagroup=True,
	restricted=False,
	signatures=False,
	min=False,
	scam=False,
	has_link=False,
	has_geo=False,
	slowmode_enabled=False,
	call_active=False,
	call_not_empty=False,
	fake=False,
	gigagroup=False,
	noforwards=False,
	join_to_send=False,
	join_request=False,
	forum=False,
	stories_hidden=False,
	stories_hidden_min=False,
	stories_unavailable=True,
	access_hash=5073239891230037431,
	username=None,
	restriction_reason=[
	],
	admin_rights=ChatAdminRights(
		change_info=True,
		post_messages=True,
		edit_messages=True,
		delete_messages=True,
		ban_users=True,
		invite_users=True,
		pin_messages=True,
		add_admins=True,
		anonymous=False,
		manage_call=True,
		other=True,
		manage_topics=True,
		post_stories=True,
		edit_stories=True,
		delete_stories=True


Abbiamo ottenuto un oggetto Channel utilizzando il link del gruppo di test.

Possiamo anche **partecipare ad un gruppo/canale pubblico**, utilizzando un oggetto `JoinChannelRequest` nel modulo `telethon.tl.functions.channels` e fornendo l'entity channel.

Per esempio, per iscriversi a`jakidale`, prima verifichiamo che tipo di entity sia: `Chat` o `Channel`, poi chiediamo la partecipazione.

In [ ]:
async with client:
    #

1038653661


In [ ]:
from telethon.tl.functions.channels import JoinChannelRequest

In [ ]:
async with client:
    #

Possiamo verificare se ora il canale/gruppo è nella lista delle conversazioni mediante il metodo `get_dialogs`.

In [ ]:
async with client:
    #

Per ogni conversazioni possiamo farci restituire il titolo (attributo `title`) e l'Id (attributo `id`), se il titolo del dialogo è `Jakidale`

In [ ]:
#

-1001038653661 Jakidale


Una volta diventati membri del canale/gruppo possiamo richiedere i messaggi di una chat/canale mediante il metodo [`iter_messages`](https://docs.telethon.dev/en/stable/modules/client.html#telethon.client.messages.MessageMethods.iter_messages). Il metodo restituisce i messaggi dal più recente al meno recente.

Se non viene specificato il parametro `limit` si ottiene l'intero lista dei messaggi di una chat.

Ottiamo tutti i messaggi nel gruppo di test.

In [ ]:
async with client:
#

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=f5fb509f-e59d-42ff-944e-a00f1bbf0efe' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>